# Deploying a Grafana/Prometheus Server

This notebook shows how to deploy a server running [Grafana](https://grafana.com/) and [Prometheus](https://prometheus.io/) in order to monitor other slices.  This example shows how to monitor a slice running Prometheus Node Exporter as shown in the companion example's [companion example notebook](node_exporter.ipynb).


## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                    
fablib.show_config();

## Create the Grafana/Prometheus Slice

Create a FABRIC node that supports Docker containers. The example using FABRIC's Docker Rocky 8 images (`docker_rocky_8`).  

The server requires network connectivity to the nodes you wish to monitor, the Grafana and Prometheus Docker containers, and a short configuration file describing the nodes to be monitored.

The example uses the `add_fabnet()` function to add a FABNetv4 network with default configuration. You can use any network type that works with your experiment. 

Grafana and Prometheus are started in Docker containers using the images supported by the vendors:

- Grafana Docker image: [grafana/grafana](https://hub.docker.com/r/grafana/grafana)
- Prometheus Docker image [prom/prometheus](https://hub.docker.com/r/prom/prometheus)


In [ ]:
slice_name = 'GrafanaServer'
site = fablib.get_random_site()

print(f"Site: {site}")

monitor_name = 'monitor'

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Monitoring Node
node = slice.add_node(name=monitor_name, site=site, cores=4, ram=16, disk=100, image='docker_rocky_8')
node.add_fabnet()

node.add_post_boot_upload_directory('node_tools','.')
node.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }} ')
node.add_post_boot_upload_directory("./monitoring", ".")
node.add_post_boot_execute("cd monitoring ; docker compose up -d")

#Submit Slice Request
slice.submit();

## Start an SSH tunnel to the Grafana Server

At this point your Grafana server is running.  You will need to create an SSH tunnel to access it. 

When you ran the [Configure Environment](../../../configure.ipynb) notebook, it created a`fabric_ssh_tunnel_tools.tgz`package that can be used to create an SSH tunnel.

- Download your custom `fabric_ssh_tunnel_tools.tgz` tarball from the main jupyter-examples folder OR the fabric_config folder.  This tarball was created when you ran the configure environment notebook. Rerun the configure environment notebook if you do not have fabric_ssh_tunnel_toold.tgz 
- Untar the tarball and put the resulting folder (`fabric_ssh_tunnel_tools`) somewhere you can access it from the command line.
- Open a terminal window. (Windows: use `powershell`) 
- Use `cd` to navigate to the `fabric_ssh_tunnel_tools` folder.
- In your terminal, run the command that results from running the following cell (leave the terminal window open).


In [ ]:
slice = fablib.get_slice(slice_name)

monitor = slice.get_node(monitor_name)

management_ip=f'{monitor.get_username()}@{monitor.get_management_ip()}'

print(f'ssh  -L 127.0.0.1:3000:127.0.0.1:3000 -i slice_key -F ssh_config {management_ip}')

## Open the Grafana Dashboard

Click this link to open your [Grafana Dashboard](http://127.0.0.1:3000)



## Configure the Grafana and Prometheus Servers

At this point your Grafana and Prometheus servers are active but are not configured to monitor anything. 

Run the following cell to generate a Prometheus configuration file for monitoring the slice you started using the [companion notebook](node_exporter.ipynb).  

This cell assumes you used the companion notebook and the target slice is connected to FABNetv4 using the `add_fabnet()` function and has a node exporter running on each node. If you provide alternative connectivity between the monitor node and the target slice, you will need to modify the config file with the correct IP addresses.   If you have multiple slices to monitor, you can add each of them to the `slices_to_monitor` list.

<div class="alert alert-block alert-warning">
<b>Important:</b> You must run the companion notebook before continuing.
</div>





In [ ]:
slice = fablib.get_slice(slice_name)

monitor = slice.get_node(monitor_name)

slices_to_monitor=['MySlice']

import yaml
config_dict = {}
with open("./monitoring/prometheus/prometheus_template.yml", "r") as f:
    config_dict = yaml.safe_load(f)
    sc = config_dict['scrape_configs']
    
    for slice_to_monitor in slices_to_monitor:
        target_slice=fablib.get_slice(slice_to_monitor)
        targets = []
        for node in target_slice.get_nodes():
            targets.append(f"{node.get_interface(network_name=f'FABNET_IPv4_{node.get_site()}').get_ip_addr()}:9100")

        new_job = {'job_name': target_slice.get_name(),
                   'static_configs': [ { 'targets': targets }]} 
        sc.append(new_job)

with open("prometheus.yml", "w") as f:
    yaml.dump(config_dict, f)

    

## View the Config File

Before uploading it to your server, you can manually edit the config file to add/remove any features you desire or customize the IP addresses to suit your experiment.




In [ ]:
!cat prometheus.yml

## Configure Prometheus

Upload the config file and restart Prometheus 

In [ ]:
monitor.upload_file("prometheus.yml", "./monitoring/prometheus/prometheus.yml")
stdout, stderr = monitor.execute("docker restart prometheus");

## View your Slice in Grafana

View your slice in your Grafana browser tab (or open a [new dashboard](http://127.0.0.1:3000)).  You will need to wait for Prometheus to connect to nodes.  You may need to reload your browser window.





## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()